In [1]:
'''
Created on 2020年6月30日

@author: gaoruiqi
'''
#-------------------------------------------------------------------
'''
程序部分PART1:机器学习

程序使用开源机器学习项目XGBoost进行机器学习。
XGBoost的核心算法思想基本：
1、不断地添加树，不断地进行特征分裂来生长一棵树，每次添加一个树，其实是学习一个新函数f(x)，去拟合上次预测的残差；
2、当训练完成得到k棵树，要预测一个样本的分数，就是根据这个样本的特征在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数；
3、最后只需要将每棵树对应的分数加起来就是该样本的预测值。

程序使用sklearn机器学习的第三方模块，对常用的机器学习方法进行了封装，包括回归(Regression)、
降维(Dimensionality Reduction)、分类(Classfication)、聚类(Clustering)等。
Sklearn具有以下特点：
1、简单高效的数据挖掘和数据分析工具；
2、让每个人能够在复杂环境中重复使用；
3、建立NumPy、Scipy、MatPlotLib之上。
'''

import xgboost as xgb#引入三方 xgboost算法库包，使用xgboost算法作若干的机器学习任务
import pandas as pd#引入三方解决数据分析任务的库包
import matplotlib.pyplot as plt#引入数据绘图包
#引入sklearn机器学习模块/库 ，交叉验证用于评估模型的预测性能
from sklearn.model_selection import train_test_split#引入 分割数据模块
from sklearn.metrics import auc, roc_curve#metrics.评价指标函数名称
from sklearn.metrics import accuracy_score

#从csv文件中拿到表格型数据结构DataFrame
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

#处理中文显示
plt.rcParams['font.sans-serif'] = [u'SimHei']
#用来正常显示负号
plt.rcParams['axes.unicode_minus'] = False
"""
定义方法：评价得分模型
"""
def evaluate_score(predict, y_true):
    '''
          通过sklearn的roc_curve函数计算false positive rate和true positive rate以及对应的thresholds
          理论上thresholds应该取遍所有的predict（即模型预测值）
    thresholds就是predict（即模型预测值)逆序排列后的结果
    '''
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_true, predict, pos_label=1)
    auc_score = auc(false_positive_rate, true_positive_rate)#计算ROC曲线下的面积
    return auc_score

train_df.head()#返回数据

y_train_all = train_df['target']
del train_df['id']
del train_df['target']
train_df.head()

#划分训练集、验证集
"""
在使用数据集训练模型之前，我们需要先将整个数据集分为训练集、验证集、测试集。
训练集是用来训练模型的，通过尝试不同的方法和思路使用训练集来训练不同的模型，
再通过验证集使用交叉验证来挑选最优的模型，通过不断的迭代来改善模型在验证集上的性能，
最后再通过测试集来评估模型的性能。如果数据集划分的好，可以提高模型的应用速度。

如果划分的不好则会大大影响模型的应用的部署，甚至可能会使得我们之后所做的工作功亏一篑。
"""
df_columns = train_df.columns.values
print('特征计数: {}'.format(len(df_columns)))

scale_pos_weight = 1
print('scale_pos_weight = ', scale_pos_weight)

"""
参数调优
通用参数
这些参数用来控制XGBoost的宏观功能。
"""
xgb_params = {
    'eta': 0.01,#通过减少每一步的权重，可以提高模型的鲁棒性。 典型值为0.01-0.2。
    'min_child_weight': 20,#决定最小叶子节点样本权重和。
    'colsample_bytree': 0.5,#和GBM中的参数相同，这个值为树的最大深度。
    'max_depth': 15,
    'subsample': 0.9,
    'lambda': 2.0,#权重的L2正则化项。
    'scale_pos_weight': scale_pos_weight,#在各类别样本十分不平衡时，把这个参数设定为一个正值，可以使算法更快收敛。
    'eval_metric': 'auc',
    'objective': 'binary:logistic',#定义需要被最小化的损失函数;二分类的逻辑回归，返回预测的概率(不是类别)
    'nthread': -1,#参数用来进行多线程控制
    'silent': 1,#当这个参数值为1时，静默模式开启，不会输出任何信息。
    'booster': 'gbtree'#基于树的模型
}

X_train, X_valid, y_train, y_valid = train_test_split(train_df, y_train_all, test_size=0.1, random_state=42)
print('训练: {}, 验证: {}, 测试: {}'.format(X_train.shape[0], X_valid.shape[0], test_df.shape[0]))

dtrain = xgb.DMatrix(X_train, y_train, feature_names=df_columns)
dvalid = xgb.DMatrix(X_valid, y_valid, feature_names=df_columns)


watchlist = [(dtrain,'train'), (dvalid,'valid')]

#模型训练
model = xgb.train(dict(xgb_params),#字典，里面包含着训练中的参数关键字和对应的值
                      dtrain,#训练的数据
                      evals=watchlist,
                      verbose_eval=50,
                      early_stopping_rounds=200,#早期停止次数假设为100，验证集的误差迭代到一定程度在100次内不能再继续降低，就停止迭代
                      num_boost_round=3000)#提升迭代的个数

#特征重要程度情况
ax = xgb.plot_importance(model)
fig = ax.figure
fig.set_size_inches(15,10)

# 预测训练
predict_train = model.predict(dtrain)#得到概率
#返回auc_score
train_auc = evaluate_score(predict_train, y_train)

# 预测验证
predict_valid = model.predict(dvalid)
#返回auc_score
valid_auc = evaluate_score(predict_valid, y_valid)

print('train auc = {:.6f} , valid auc = {:.6f}\n'.format(train_auc, valid_auc))

#ROC 曲线,真正率（TPR）和假正率（FPR）
fpr, tpr, _ = roc_curve(y_valid, predict_valid)
roc_auc = auc(fpr, tpr)#曲线下的面积
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('假阳性率')
plt.ylabel('真阳性率')
plt.title('ROC 曲线')
plt.legend(loc="lower right")
plt.show()

#交叉训练选取最佳迭代次数
print('cv训练选择最佳次数')
dtrain_all = xgb.DMatrix(train_df.values, y_train_all, feature_names=df_columns)
#进行交叉验证
cv_result = xgb.cv(dict(xgb_params),
                   dtrain_all,
                   num_boost_round=4000,
                   early_stopping_rounds=100,
                   verbose_eval=100,
                   show_stdv=False,
                   )
best_num_boost_rounds = len(cv_result)
mean_train_logloss = cv_result.loc[best_num_boost_rounds-11 : best_num_boost_rounds-1, 'train-auc-mean'].mean()
mean_test_logloss = cv_result.loc[best_num_boost_rounds-11 : best_num_boost_rounds-1, 'test-auc-mean'].mean()
print('最佳回合数 = {}'.format(best_num_boost_rounds))

print('mean_train_auc = {:.7f} , mean_test_auc = {:.7f}\n'.format(mean_train_logloss, mean_test_logloss))

#利用最佳迭代次数，再次利用全量数据训练模型
print('预测测试和提交的总数据集的训练')
model = xgb.train(dict(xgb_params),
                  dtrain_all,
                  num_boost_round=best_num_boost_rounds)
# predict validate
predict_valid = model.predict(dvalid)
valid_auc = evaluate_score(predict_valid, y_valid)
print('预测的验证集 AUC 指标：', valid_auc)


#新用户预警
test_df.head()
test_ground_truth = test_df['target']
del test_df['target']
del test_df['id']
dtest = xgb.DMatrix(test_df, feature_names=df_columns)
predict_test = model.predict(dtest)
predict_test_label = predict_test > 0.01
acc = accuracy_score(predict_test_label, test_ground_truth)
print('新用户测试集预测准确率：', acc)
test_auc = evaluate_score(predict_test, test_ground_truth)
print('新用户测试集 AUC 指标：', test_auc)


'''
程序部分PART2:数据分析

'''

import numpy as np#引入三方维度数组与矩阵运算的库包
import seaborn as sns#引入三方图形可视化的库包
import os#引入内建的处理文件和目录的模块
import gc#引入内建的垃圾回收模块

#处理中文显示
plt.rcParams['font.sans-serif'] = [u'SimHei']
#用来正常显示负号
plt.rcParams['axes.unicode_minus'] = False
#指定随机数生成时所用算法开始的整数值
np.random.seed(7)
#将csv文件读入并转化为数据框形式
cr_data_df = pd.read_csv("test.csv")

#读取矩阵长度，如shape[0]是读取矩阵第一维的长度
print("数据集：", cr_data_df.shape)
#读取前五行数据，观察一下数据读取是否准确
cr_data_df.head()
print(cr_data_df.head())
print(cr_data_df.describe())

#违约用户性别分布
plt.figure(figsize=(10,6))#表示figure 的大小为宽、长（单位为inch）
#使用normalize=True参数进行计数排序得出占比，赋值给cr_data_df的gender
tmp = cr_data_df["gender"].value_counts(normalize=True)#从train.cvs中拿到性别数据
#返回一个包含figure和axes对象的元组
_, ax = plt.subplots()
sns.barplot(x=tmp.index.tolist(), y=tmp.values, ci=None, ax=ax)#按坐标绘制条形图
plt.xlabel('用户性别')
plt.ylabel('违约用户数量')
plt.title('违约用户性别分布')
for i, j in enumerate(tmp.sort_index()):#根据列数据/行数据排序
    ax.text(i, j, round(j, 5), ha="center", fontsize=12) #可视化text()函数

del tmp#删除变量tmp，解除tmp的引用  
gc.collect()#清理内存
plt.show()#绘制用户性别分布条状图

#违约用户年龄分布
plt.figure(figsize=(20,6))
# 对 年龄 age 维度进行统计不同值出现的个数，数值进行归一化处理
tmp = cr_data_df["age"].value_counts(normalize=True)
ax = plt.subplot(111)#单个整数编码的子绘图网格参数,“111”表示“1×1网格，第一子图
sns.barplot(x=tmp.index.tolist(), y=tmp.values, ci=None, ax=ax)#按坐标绘制条形图
plt.xlabel('用户年龄')
plt.ylabel('违约用户数量')
plt.title('违约用户年龄分布')
for i, j in enumerate(tmp.sort_index()):#根据列数据/行数据排序
    ax.text(i, j, round(j, 5), ha="center", fontsize=12)#可视化text()函数

del tmp
gc.collect()#清理内存
plt.show()#

#违约用户教育程度分布
plt.figure(figsize=(20,6))
# 对 教育程度 edu 维度进行统计不同值出现的个数，数值进行归一化处理
tmp = cr_data_df["edu"].value_counts(normalize=True)
ax = plt.subplot(111)#单个整数编码的子绘图网格参数,“111”表示“1×1网格，第一子图
sns.barplot(x=tmp.index.tolist(), y=tmp.values, ci=None, ax=ax)#按坐标绘制条形图
plt.xlabel('用户教育程度')
plt.ylabel('违约用户数量')
plt.title('违约用户教育程度分布')
for i, j in enumerate(tmp.sort_index()):#根据列数据/行数据排序
    ax.text(i, j, round(j, 5), ha="center", fontsize=12)#可视化text()函数

del tmp
gc.collect()#清理内存
plt.show()


# 不同借贷产品类型的违约比例分布 将数据集按照借贷产品类型进行分组统计，分别计算不同组违约的比例
plt.figure(figsize=(20,6))
tmp = cr_data_df.groupby("loanProduct")["target"].mean().to_frame("ratio").reset_index()
#返回一个包含figure和axes对象的元组
_, ax = plt.subplots()
sns.barplot(x="loanProduct", y="ratio", ci=None, data=tmp, ax=ax)  # DataFrame，数组或数组列表，可选
plt.xlabel('借贷产品类型')
plt.ylabel('违约比例')
plt.title('不同借贷产品类型违约比例分布')
for i, j in enumerate(tmp["ratio"]):#根据列数据/行数据排序
    ax.text(i, j, round(j, 5), ha="center", fontsize=12)

del tmp
gc.collect()#清理内存   

#预授信金额分布
plt.figure(figsize=(15, 5))
ax = plt.subplot(121)
sns.kdeplot(cr_data_df[cr_data_df['target'] == 1]['lmt'])
ax.set_title('正常用户')
ax = plt.subplot(122)
sns.kdeplot(cr_data_df[cr_data_df['target'] == 0]['lmt'])
ax.set_title('风险用户')
plt.show()

ModuleNotFoundError: No module named 'xgboost'